## Parsiranje ulaznih Excel/CSV fajlova

### Izlazni entiteti:
- Departmani
- Katedre
- predavaci_df
- Prostorije
- Studijski programi
- Studentske grupe
- Predmeti
- Realizacija

### Ulazni fajlovi:
- ```Oznake departmana i katedri.xlsx```
- ```nastavnik - katedra.xlsx```
- ```dekanat.csv```
- ```sefovi_katedri.csv```
- ```sefovi_departmana.csv```
- ```Realizacija.xlsx```
- ```Grupe_Stud_sluzba_2022Z_OSP_proba.xlsx```
- ```Prostorije i organizacija.xlsx```


In [ ]:
import pandas as pd
import uuid

In [ ]:
from model_parser import *

#### (E) Ekstrakcija podataka o **departmanima i katedrama**

##### Primer ulaznih podataka
- Excel
- postoji header red
- ```data/Oznake departmana i katedri.xlsx```

![Departmani i katedre](../docs/excel_sample_data_departmani_katedre.png)

In [ ]:
def extract_departmani_katedre(
        file_path: str ='../data/Oznake departmana i katedri.xlsx',
        sheet_name: str ='Departmani i katedre (2)'
) -> tuple[list[Departman], list[Katedra]]:

    # relevant columns
    column_names = ['dep_sluzba_oznaka', 'dep_oznaka', 'dep_naziv', 'kat_oznaka', 'kat_sluzba_oznaka', 'kat_naziv']
    
    dep_kat_df = pd.read_excel(file_path, sheet_name=sheet_name, names=column_names)
    dep_kat_df.fillna(method='ffill', inplace=True)

    # ex. '*015' -> 15
    dep_kat_df['dep_oznaka'] = pd.to_numeric(dep_kat_df['dep_oznaka'].map(lambda x: x.lstrip('*0')))

    # departmani
    dep_df = dep_kat_df[['dep_sluzba_oznaka', 'dep_oznaka', 'dep_naziv']]
    dep_df.columns = ['ssluzbaOznaka', 'oznaka', 'naziv']
    dep_df = dep_df.drop_duplicates()

    # create objects
    departman_list = [(Departman(str(uuid.uuid4()), row.oznaka, int(row.ssluzbaOznaka), row.naziv)) for _, row in dep_df.iterrows()]

    # katedre
    kat_df = dep_kat_df[['dep_oznaka', 'kat_oznaka', 'kat_sluzba_oznaka', 'kat_naziv']].copy()
    kat_df.columns = ['depOznaka', 'oznaka', 'ssluzbaOznaka', 'naziv']
    # remove all invalid
    kat_df = kat_df[kat_df.ssluzbaOznaka != '***']
    kat_df['naziv'] = kat_df['naziv'].str.strip()

    katedra_list = []
    for _, row in kat_df.iterrows():
        # find parent departman
        departman = next((x for x in departman_list if x.oznaka == row.depOznaka), None)
        katedra_list.append(Katedra(str(uuid.uuid4()), row.oznaka, row.ssluzbaOznaka, row.naziv, departman.id))

    return departman_list, katedra_list

#### (E) Ekstrakcija podataka o **predavacima**

##### Problem
- Nema informacije o tituli (dr, mr,...)

##### Primer ulaznih podataka
- Predavaci i pripadnost katedri
    - Excel
    - **ne** postoji header red 
    - ```data/nastavnik - katedra.xlsx```
    
![Predavaci i pripadnost katedri](../docs/excel_sample_data_predavaci.png)
- Pripadnici dekanata/Šefovi departmana/Šefovi katedri
    - CSV
    - postoji header red
    - ```data/dekanat.csv```
    - ```data/sefovi_katedri.csv```
    - ```data/sefovi_departmana.csv```

![Sefovi departmana](../docs/csv_sefovi_departmana.png)


In [ ]:
def extract_predavaci(
        departman_list: list[Departman],
        katedra_list: list[Katedra],
        predavaci_file_path: str = '../data/nastavnik - katedra.xlsx',
        dekanat_file_path: str = '../data/dekanat.csv',
        sefovi_katedre_file_path: str = '../data/sefovi_katedri.csv',
        sefovi_departmani_file_path: str = '../data/sefovi_departmana.csv',
) -> list[Predavac]:
    # relevant columns
    column_names = ['oznaka', 'prezime', 'ime', 'kat_sluzba_oznaka']
    columns = 'B:E'
    
    # predavaci
    predavaci_df = pd.read_excel(predavaci_file_path, header=None, names=column_names, usecols=columns)
    predavaci_df = predavaci_df.drop_duplicates()
    
    # dekanat
    dekanat = pd.read_csv(dekanat_file_path)
    predavaci_df = predavaci_df.merge(dekanat, on='oznaka', how='left', suffixes=('', '_dek'))
    predavaci_df.loc[pd.isna(predavaci_df['ime_dek']), 'dekanat'] = False
    predavaci_df['dekanat'].fillna(True, inplace=True)
    predavaci_df.drop(['ime_dek', 'prezime_dek'], axis=1, inplace=True)

    # sefovi katedre
    sef_kat = pd.read_csv(sefovi_katedre_file_path)
    predavaci_df = predavaci_df.merge(sef_kat, on='oznaka', how='left', suffixes=('', '_kat'))
    predavaci_df.loc[pd.isna(predavaci_df['ime_kat']), 'sefKat'] = False
    predavaci_df['sefKat'].fillna(True, inplace=True)
    predavaci_df.drop(['ime_kat', 'prezime_kat'], axis=1, inplace=True)

    # sefovi departmana
    sef_dep = pd.read_csv(sefovi_departmani_file_path)
    predavaci_df = predavaci_df.merge(sef_dep, on='oznaka', how='left', suffixes=('', '_dep'))
    predavaci_df.loc[pd.isna(predavaci_df['ime_dep']), 'sefDep'] = False
    predavaci_df['sefDep'].fillna(True, inplace=True)
    predavaci_df.drop(['ime_dep', 'prezime_dep'], axis=1, inplace=True)

    # organizacija fakulteta (sef katedre ili sef departmana)
    predavaci_df['organizacijaFakulteta'] = predavaci_df.apply(lambda row: row.sefKat or row.sefDep, axis=1)
    predavaci_df.drop(['sefKat', 'sefDep'], axis=1, inplace=True)

    predavac_list = []
    for _, row in predavaci_df.iterrows():
        katedra = next((x for x in katedra_list if x.ssluzbaOznaka == row.kat_sluzba_oznaka), None)
        departman = next((x for x in departman_list if x.ssluzbaOznaka == row.kat_sluzba_oznaka), None)
        if katedra is None:
            katedra = departman
        predavac_list.append(
            Predavac(str(uuid.uuid4()), row.oznaka, row.ime, row.prezime, row.organizacijaFakulteta, row.dekanat, katedra.id))
    
    return predavac_list

#### (E) Ekstrakcija podataka o **studijskim programima**

##### Primer ulaznih podataka
- Realizacija nastave u školskoj godini
    - Excel
    - postoji header red
    - ```data/Realizacija.xlsx```

![Realizacija nastave u školskoj godini](../docs/excel_realizacija.png)

- Primer relevantnih kolona

![Realizacija - studijski programi](../docs/realizacija_studijski_programi.png)


In [ ]:
def extract_studijski_programi(
        file_path: str = '../data/Realizacija.xlsx'
) -> list[StudijskiProgram]:
    # relevant columns
    columns = 'A,B,D,F'
    column_names = ['stepen', 'nivo', 'oznaka', 'naziv']
    stud_programi_df = pd.read_excel(file_path, names=column_names, usecols=columns)
    stud_programi_df = stud_programi_df.drop_duplicates()

    return [(StudijskiProgram(str(uuid.uuid4()), row.stepen, row.nivo, row.oznaka, row.naziv)) 
            for _, row in stud_programi_df.iterrows()]

#### (E) Ekstrakcija podataka o **predmetima**

##### Primer ulaznih podataka
- Realizacija nastave u školskoj godini
    - Excel
    - postoji header red
    - ```data/Realizacija.xlsx```

![Realizacija nastave u školskoj godini](../docs/excel_realizacija.png)

- Primer relevantnih kolona

![Realizacija - predmeti](../docs/realizacija_predmeti.png)


In [ ]:
def extract_predmeti(
        studijski_programi_list: list[StudijskiProgram],
        file_path: str = '../data/Realizacija.xlsx'
) -> list[Predmet]:
    # relevant columns
    columns = 'A,B,D,G:K,M,S:U,W'
    column_names = ['stepen', 'nivo', 'oznaka_sp', 'godina', 'semestar', 'plan', 'oznaka', 'naziv', 'tipovi_nastave', 'pred', 'aud', 'lab', 'ostalo']
    predmeti_df = pd.read_excel(file_path, names=column_names, usecols=columns)

    predmeti_df['oznaka'] = predmeti_df.apply(lambda row: row.oznaka.upper(), axis=1)
    predmeti_df = predmeti_df.drop_duplicates()

    # izbaci sve koji se ne odrzavaju (izrade radova)
    predmeti_df = predmeti_df[~((predmeti_df.pred == 0) & (predmeti_df.aud == 0) & (predmeti_df.lab == 0))]
    # izbaciti sve koji su na PHD studijama
    predmeti_df = predmeti_df[~(predmeti_df.stepen == 3)]
    # izbaciti sve sa 'Studenti na razmeni' studijskog programa
    predmeti_df = predmeti_df[~(predmeti_df.oznaka_sp == 'FTN')]

    predmeti_df = predmeti_df.groupby(['stepen', 'nivo', 'oznaka_sp', 'godina', 'semestar', 'plan', 'oznaka', 'naziv', 'pred', 'aud', 'lab', 'ostalo'])['tipovi_nastave'].apply('|'.join).reset_index()
    
    # dodavanje broja casova ostalog na laboratorijske (ako postoje), ako ne, dodaj na auditorne (ako postoje)
    predmeti_df['lab'] = predmeti_df.apply(lambda row: row.lab + row.ostalo if row.lab != 0 else 0, axis=1)
    predmeti_df['aud'] = predmeti_df.apply(lambda row: row.aud + row.ostalo if row.aud != 0 and row.lab == 0 else 0, axis=1)

    predmet_list = []
    for _, row in predmeti_df.iterrows():
        stud_program = next((x for x in studijski_programi_list if x.stepen == row.stepen and x.nivo == row.nivo and x.oznaka == row.oznaka_sp), None)
        predmet_list.append(Predmet(str(uuid.uuid4()), row.oznaka, row.plan, row.naziv, row.godina, row.semestar, row.tipovi_nastave, row.pred, row.aud, row.lab, stud_program))
    
    return predmet_list

#### (E) Ekstrakcija podataka o **studentskim grupama**

##### Primer ulaznih podataka
- Studentske grupe - trenutni broj
    - Excel
    - postoji header red
    - ```data/Grupe_Stud_sluzba_2022Z_OSP_proba.xlsx```
    - raznovrsni zapisi broja studenata po grupi, podržani sledeći:
        - 15
        - gr 15 sa 12st
        - Gr. 1 po 32 st.
        - po 15st., Maket. gr.1,2,3,4 i Principi.  Gr.5,6,7,8
        - Grupe 31,32  po 15 studenata
        - gr 1,2,3,4 po 16st i gr 5,6 po 8st
        - Gr. od 1 do 5  po 14 st.,
        - Grupe 11- 8 st
        - Grupe 21 do 28 po 11 st.

![Studentske grupe](../docs/studentske_grupe.png)


##### Pomoćne funkcije za transformaciju podataka

- Transformacija rimskih u arapske brojeve
- Transformacija broja semestra (6) u godinu (3) i oznaku (L/Z) 
- Transformacija broja studenata po grupi

In [ ]:
import re
from parser_utils import roman_to_arab, semestar_to_godina, semestar_to_oznaka

In [ ]:
def broj_stud_po_gr(text: str) -> int|str:
    if type(text) == int:
        return text
    # ex. "14 st"
    br = re.findall(r"(\d+)\s?st", text)
    if len(br) == 1:
        return int(br[0])
    # ex. "po 15st., Maket. gr.1,2,3,4 i Principi.  Gr.5,6,7,8"
    grupe_broj = []
    grupe = text.split('i')
    for i in range(len(br)):
        br_grupa = len(grupe[i].split(','))
        grupe_broj.append(str(br_grupa) + ' po ' + br[i])
    # return format: broj_grupa,broj_studenata|broj_grupa,broj_studenata
    return ('|').join(grupe_broj)

In [ ]:
def extract_studentske_grupe(
        studijski_programi_list: list[StudijskiProgram],
        file_path: str = '../data/Grupe_Stud_sluzba_2022Z_OSP_proba.xlsx',
        sheet_name: str = 'Trenutno - letnji'
) -> list[StudentskaGrupa]:
    # relevant columns
    columns = 'C:I'
    column_names = ['semestar_rimski', 'stepen', 'nivo', 'oznaka_sp', 'br_stud', 'br_gr', 'br_stud_po_gr_str']

    stud_grupe = pd.read_excel(file_path, sheet_name=sheet_name, names=column_names, usecols=columns)

    # transform broj semsestra
    stud_grupe['semestar_arapski'] = stud_grupe.apply(lambda row: roman_to_arab(row.semestar_rimski), axis=1)
    stud_grupe['godina'] = stud_grupe.apply(lambda row: semestar_to_godina(row.semestar_arapski), axis=1)
    stud_grupe['semestar'] = stud_grupe.apply(lambda row: semestar_to_oznaka(row.semestar_arapski), axis=1)
    stud_grupe.drop(['semestar_rimski', 'semestar_arapski'], axis=1, inplace=True)
    # transform broj studenata po grupi
    stud_grupe['br_stud_po_gr'] = stud_grupe.apply(lambda row: broj_stud_po_gr(row.br_stud_po_gr_str), axis=1)

    studentska_grupa_list = []
    for _, row in stud_grupe.iterrows():
        stud_prog = next((x for x in studijski_programi_list if x.stepen == row.stepen and x.nivo == row.nivo and x.oznaka == row.oznaka_sp), None)

        if row.br_gr == 0 or row.br_stud == 0:
            continue
        if stud_prog is not None:
            if type(row.br_stud_po_gr) != int:
                # ex. "4 po 16|2 po 8"
                grupe = row.br_stud_po_gr.split('|')
                j = 0
                for grupa in grupe:
                    broj_studenata_grupa = grupa.split(' po ')
                    broj_grupa = int(broj_studenata_grupa[0])
                    broj_studenata_po_grupi = int(broj_studenata_grupa[1])
                    for _ in range(broj_grupa):
                        studentska_grupa_list.append(StudentskaGrupa(str(uuid.uuid4()), j+1, row.godina, row.semestar, broj_studenata_po_grupi, stud_prog.id))
                        j += 1
            else:
                for i in range(row.br_gr):
                    studentska_grupa_list.append(StudentskaGrupa(str(uuid.uuid4()), i+1, row.godina, row.semestar, row.br_stud_po_gr, stud_prog.id))
        else:
            print(row.oznaka_sp)
    return studentska_grupa_list

#### (E) Ekstrakcija podataka o **prostorijama**

##### Primer ulaznih podataka
- Prostorije - u svim objektima fakulteta
    - Excel
    - postoji header red
    - ```data/Prostorije i organizacija.xlsx```
    - pripadnost katedri/departmanu
        - opšta
        - sekundarna pripadnost
    - primarna i sekundarna namena prostorije
        - auditorna
        - računarska
        - laboratorijska
    - posebne učionice vezane direktno za predmete
        - hemija/fizika

![Prostorije](../docs/prostorije.png)


In [ ]:
def map_prostorija_ownership(
        org_jedinice_list: list[Departman|Katedra],
        ownership: str
) -> list[Departman|Katedra]|None:
    if ownership != 'nan':
        org_jed_ids = ownership.split('|')
        org_jed_list = []
        for org_jed_id in org_jed_ids:
            org_jed_list.append(next((x.id for x in org_jedinice_list if x.oznaka == int(org_jed_id)), None))
    else:
        # opsta prostorija
        org_jed_list = None
    return org_jed_list


In [ ]:
def extract_prostorije(
        departmani_list: list[Departman],
        katedre_list: list[Katedra],
        file_path: str = '../data/Prostorije i organizacija.xlsx',
        sheet_name: str = 'Prostorije-upotreba'
) -> list[Prostorija]:
    # relevant columns
    columns = 'A,B,D,G,H,J,K'
    column_names = ['oznaka', 'kapacitet', 'tip', 'org_jed_id', 'sek_tip', 'sek_org_jed_id', 'oznaka_sistem']
    
    prostorije = pd.read_excel(file_path, sheet_name=sheet_name, names=column_names, usecols=columns)
    
    # transform data
    prostorije['org_jed_id'] = prostorije['org_jed_id'].astype(str)
    prostorije['sek_org_jed_id'] = prostorije['sek_org_jed_id'].astype(str)
    prostorije['sek_tip'] = prostorije['sek_tip'].astype(str)
    prostorije['oznaka_sistem'] = prostorije['oznaka_sistem'].astype(str)

    prostorija_list = []
    org_jedinice_list = departmani_list + katedre_list

    for _, row in prostorije.iterrows():
        # dodavanje pripadnosti primarnoj organizacionoj jedinici
        primary_ownership = map_prostorija_ownership(org_jedinice_list, row.org_jed_id)
        secondary_ownership = map_prostorija_ownership(org_jedinice_list, row.sek_org_jed_id)

        # sekundarna namena
        sek_tip = None
        if row.sek_tip != 'nan':
            sek_tip = row.sek_tip

        oznaka_sistem = None
        if row.oznaka_sistem != 'nan':
            oznaka_sistem = row.oznaka_sistem
        prostorija_list.append(Prostorija(str(uuid.uuid4()), row.oznaka, row.tip, row.kapacitet, primary_ownership, oznaka_sistem, sek_tip, secondary_ownership))
    
    return prostorija_list

#### Generisanje Dana i TimeGrain-ova
- dani: ponedeljak, utorak, sreda, četvrtak, petak, subota
- time grain: 15-minutni period u danu počev od 7 ujutru

In [ ]:
def generate_days_and_time_grains(
        day_num: int = 5,
        start_hour: int = 9,
        hour_per_day_num: int = 11
) -> tuple[list[Dan], list[TimeGrain]]:
    time_grain_per_day_num = 4 * hour_per_day_num + 1

    days = []
    time_grains = []
    for i in range(day_num):
        day = Dan(i, i)
        days.append(day)
        start_time = start_hour * 60
        for j in range(time_grain_per_day_num):
            time_grains.append(TimeGrain(str(uuid.uuid4()), i * time_grain_per_day_num + j, start_time + j * 15, day.id))
    
    return days, time_grains

#### Izvršavanje

In [ ]:
# Organizacija
departmani, katedre = extract_departmani_katedre()
print(departmani[0])
print(katedre[0])

# Predavaci - katedra
predavaci = extract_predavaci(departmani, katedre)
print(predavaci[0])

# Realizacija
studijski_programi = extract_studijski_programi()
print(studijski_programi[0])
predmeti = extract_predmeti(studijski_programi)
print(predmeti[0])

# Grupe
studentske_grupe = extract_studentske_grupe(studijski_programi)
print(studentske_grupe[0])

# Prostorije
prostorije = extract_prostorije(departmani, katedre)
print(prostorije[0])

# Vreme
dani, time_grains = generate_days_and_time_grains()
print(dani[0])
print(time_grains[0])

schedule = MeetingSchedule('Z', departmani, katedre, predmeti, studijski_programi, prostorije, predavaci, studentske_grupe, dani, time_grains)
ReadWrite.write_to_file(schedule, '1_svi_fajlovi')